In [1]:
import gsd
import gsd.hoomd
import MDAnalysis as mda
import numpy as np
from MDAnalysis.analysis import polymer
import matplotlib.pyplot as plt
import scipy

In [2]:
def time_persistence_length(
    gsd_file, select_atoms_arg, window_size, start=0, stop=1
):
    """Performs time-average sampling of persistence length using MDAnalysis.

    See:
    https://docs.mdanalysis.org/stable/documentation_pages/analysis/polymer.html

    Parameters
    ----------
    gsd_file : str; required
        Path to a gsd_file
    slect_atoms_arg : str; required
        Valid argument to MDAnalysis.universe.select_atoms
    window_size : int; required
        The number of frames to use in
    start: int; optional; default 0
        The frame index of the trajectory to begin with
    stop: int; optional; default -1
        The frame index of the trajectory to end with
    """
    lp_results = []
    sampling_windows = np.arange(start, stop + 1, window_size)
    for idx, frame in enumerate(sampling_windows):
        try:
            u = mda.Universe(gsd_file)
            chains = u.atoms.fragments
            backbones = [
                chain.select_atoms(select_atoms_arg) for chain in chains
            ]
            sorted_backbones = [polymer.sort_backbone(bb) for bb in backbones]
            _pl = polymer.PersistenceLength(sorted_backbones)
            pl = _pl.run(start=frame, stop=sampling_windows[idx + 1] - 1)
            lp_results.append(pl.results.lp)
        except IndexError:
            pass
    return np.mean(lp_results), np.std(lp_results)

In [3]:
def exp_seg_persistence_length(filepath,start=0,stop=-1,interval=1):
    """
    filepath needs to be a format in which you can
    create an mdanalysis universe from, we mostly use gsd files
    """
    u = mda.Universe(topology=filepath)

    """rewrite atom indices"""
    bond_indices = []
    particle_index = [0]
    for i in range(len(u.bonds)):
        a = u.bonds[i].atoms.indices
        bond_indices.append(list(a))
        if particle_index[-1] in bond_indices[i]:
            atom1 = bond_indices[i][0]
            atom2 = bond_indices[i][1]
            if atom1 not in particle_index:
                particle_index.append(atom1)
            if atom2 not in particle_index:
                particle_index.append(atom2)

    """create bonds list"""
    av = []
    bond_len = []
    for t in u.trajectory[start:stop:interval]:
        particle_positions = []
        bonds = []
        unit_bonds = []
        bond_lengths = []
        angles = []

        for i in particle_index:
            pos = t.positions[i]
            particle_positions.append(pos)
        for i in range(len(u.bonds)):
            b = particle_positions[i+1]-particle_positions[i]
            bonds.append(b)
            l2 = t.dimensions[0]/2
            for i,b in enumerate(bonds):
                for j,x in enumerate(b):
                    if x>l2:
                        bonds[i][j] = x-l2*2
                    if x<-l2:
                        bonds[i][j] = x+l2*2
            a = b/np.linalg.norm(b)
            unit_bonds.append(a)
            length = np.linalg.norm(b)
            bond_lengths.append(length)
            #l_b = np.mean(bond_lengths)
        bond_len.append(bond_lengths)

        for i in range(len(unit_bonds)-1):
            b1 = unit_bonds[0]
            b2 = unit_bonds[0+i]
            dot_product = np.dot(b1,b2)
            angles.append(dot_product)

        n=len(u.atoms)
        n_frames = 1
        n_chains = 1
        norm = np.linspace(n - 1, 1, n - 1)
        norm *= n_chains * n_frames
        auto = autocorr1D(angles)
        av.append(auto)

    '''average the data from trajectories together'''
    sums = []
    for j in range(len(av[0])):
        k = []
        for i in range(len(av)):
            a = av[i][j]
            k.append(a)
        sum = np.sum(k)
        sums.append(sum)
    l_b = np.average(bond_len)
    result = [x/len(av) for x in sums]
    x = [i for i in range(len(sums))]

    '''set negative results to 0'''
    for r in range(len(result)):
        if result[r] < 0:
            result[r] = 0
    def expfunc(x, a):
        return np.exp(-x/a)

    exp_coeff = scipy.optimize.curve_fit(expfunc,x,result)[0][0]

    l_p = exp_coeff * l_b

    fit = np.exp(-(x/exp_coeff))

    return l_p, l_b, x, result, fit

In [4]:
traj = 'cg-pps-trajectory-50.gsd'
time_p_l, time_p_l_std = time_persistence_length(traj,select_atoms_arg='name A',window_size=10)
print(time_p_l,'+/-',time_p_l_std)

nan +/- nan


/Users/stephaniemccallum/miniforge3/envs/mupt-analysis/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/stephaniemccallum/miniforge3/envs/mupt-analysis/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/stephaniemccallum/miniforge3/envs/mupt-analysis/lib/python3.12/site-packages/numpy/_core/_methods.py:222: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/stephaniemccallum/miniforge3/envs/mupt-analysis/lib/python3.12/site-packages/numpy/_core/_methods.py:180: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/stephaniemccallum/miniforge3/envs/mupt-analysis/lib/python3.12/site-packages/numpy/_core/_methods.py:214: RuntimeWarning: invalid value e

In [18]:
exp_p_l, bond_l, x, result, fit = exp_seg_persistence_length(traj)

IndexError: list index out of range